# Word2Vec - Exercise

## Necessary imports

In order to handle the data properly we have to import the data and the modules we need:

In [80]:
# modules
import pandas as pd
import numpy as np
import re
import nltk
from gensim.models import Word2Vec

First of all, you need to download the data set "tweets.csv" from the GitHub repository https://github.com/assenmacher-mat/nlp_notebooks.

__If you are running this notebook on colab (https://colab.research.google.com/), you also need to run the next chunk in order to upload the data to colab.  
Choose it in the upload window and in it will be available on colab from now on.__  
(If you are running this notebook locally on your machine, you can skip the execution of this chunk)

In [124]:
from google.colab import files
uploaded = files.upload()

### Import the data set

__If you are running this notebook locally on your machine, you might need to adjust the path (depending on where you've saved the data).__  
(If you are running this notebook on colab, you can can leave the path unchanged)

In [2]:
tweet_data = pd.read_csv("trump.csv")

### Next, just have a look at the data set in order to see what's inside

In [3]:
tweet_data.loc[:3,:]

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,At the request of @SenThomTillis I have declar...,10-04-2019 21:59:44,8562,36356,False,1180241114403610626
1,Twitter for iPhone,Under my Administration Medicare Advantage pre...,10-04-2019 21:57:17,15248,54729,False,1180240498478534658
2,Twitter for iPhone,WOW this is big stuff! https://t.co/H12yxMfua3,10-04-2019 19:46:59,15655,50526,False,1180207709985165313
3,Twitter for iPhone,“I think it’s outrages that a Whistleblower is...,10-04-2019 14:12:23,19441,73966,False,1180123504924151809


### Extract the tweets to a list of texts:

In [4]:
tweets_raw = [tweet for tweet in list(tweet_data.text)]

### Display one exemplary tweet:

In [5]:
print(tweets_raw[1])

Under my Administration Medicare Advantage premiums next year will be their lowest in the last 13 years. We are providing GREAT healthcare to our Seniors. We cannot let the radical socialists take that away through Medicare for All!


### Perform the basic preprocessing steps before we continue:
    - everything to lowercase
    - expand contractions
    - delete url adresses
    - delete other unwanted tokens
    - tokenizing

In [179]:
tweets = [doc.lower() for doc in tweets_raw]

In [191]:
conts = [(r"don't", "do not"), (r"isn't", "is not")]
def expand(text, contractions = conts):
    for c in contractions:
        t = re.sub(c[0], c[1], text)
    return t

In [190]:
print(tweets[33])
print(expand(tweets[33]))

leader mccarthy we look forward to you soon becoming speaker of the house. the do nothing dems don’t have a chance! https://t.co/uwpdgjg99f
leader mccarthy we look forward to you soon becoming speaker of the house. the do nothing dems don’t have a chance! https://t.co/uwpdgjg99f


### Print a list of unique tokens that are at the moment present in our corpus
### Based on this, we can identify which tokens occur the we potentially want to exclude

In [155]:
print(sorted(set(nltk.word_tokenize(" ".join(tweets)))))

['!', '#', '$', '%', '&', "'", "''", "'case", "'collusion", "'could", "'crisis", "'forgotten", "'god", "'right", "'s", "'spying", "'ve", '(', ')', '+', '-', '--', '.', '..', '...', '..again', '..all', '..also', '..amounts', '..are', '..between', '..breaking', '..but', '..call', '..came', '..chairman', '..comcast', '..congresswomen', '..deferral', '..despite', '..if', '..mexico', '..much', '..my', '..news', '..nice', '..not', '..now', '..on', '..other', '..saying', '..shouting', '..sorry', '..spread', '..thank', '..that', '..the', '..there', '..this', '..to', '..tv', '..united', '..was', '..we', '..who', '..why', '..willing', '..years', '.33000', '.a', '.about', '.adds', '.after', '.again', '.agricultural', '.alabama', '.alex', '.all', '.almost', '.also', '.alternative', '.amendment.', '.amounts', '.an', '.and', '.another', '.are', '.as', '.asking', '.at', '.average', '.back', '.bad', '.based', '.be', '.became', '.because', '.best', '.better', '.between', '.big', '.border', '.brought', 

In [164]:
tweets = [re.sub(r"https://.*|“|”|@", "", doc) for doc in tweets]

In [165]:
tweets = [re.sub(r"[\)\(\.\,;:!?\+\-\_\#\'\*\§\$\%\&]", "", doc) for doc in tweets]

In [166]:
tweets = [nltk.word_tokenize(doc) for doc in tweets]

In [76]:
print(sorted(set([word for tweet in tweets for word in tweet])))

["''", '/south', '0', '00', '00p', '00pm', '03', '09', '1', '1/1024th', '1/2', '10', '100', '1000', '1000/24th', '10000', '100000', '1000000', '1036', '104th', '105', '107', '10th', '11', '11000000', '1112', '11th', '12', '122', '125th', '12th', '13', '133000', '138', '14', '145', '149', '14th', '15', '150', '1500', '150th', '157005000', '158000000', '15th', '16', '160th', '17', '170', '17000', '170000', '18', '180', '1800', '187', '18959495168', '18th', '19', '1951', '196000', '1969', '1970s', '1972', '1976', '1977', '1980', '1984', '1990', '1994', '1997', '1998', '19th', '1st', '2', '20', '200', '2000', '20000', '2001', '2002', '2005', '2010', '2011', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2020takebackthehouse', '2021', '2024', '205', '20th', '21', '21st', '22', '223', '23', '232', '24', '24/7/365', '240', '25', '250', '25000', '250m', '25772342', '25k', '25th', '26', '263000', '266000', '27', '27000', '27th', '28', '284', '292000', '2a', '2nd', '2yrs', '3', '30', '

# After all, we can finally start with the modeling part!  
(If you want to have a look at the help page, just execute the following chunk)

In [1]:
help(Word2Vec)

NameError: name 'Word2Vec' is not defined

### First, we determine the number of CPUs that are available on our machine  
(The more cores are available, the faster we can train our model)

In [78]:
import multiprocessing
cpus = multiprocessing.cpu_count()
print(cpus)

8


### Set up the model parameters

In [92]:
w2v_model = Word2Vec(sg = 0, cbow_mean = 1, size = 100, alpha = 0.025, min_alpha = 0.0001, 
                     window = 5, min_count = 5, sample = 0.001, negative = 5, workers = cpus - 1)

### Initialize the model with our twitter data:

In [93]:
w2v_model.build_vocab(sentences = tweets, update = False, progress_per = 10000)

### Train our model:  
(Hint: If you want to compre the runtime of the model for different number of cores or epochs, just put "%timeit" in front of the command  
 in the next chunk. You will then get an evaluation of how long the process takes.)

In [115]:
w2v_model.train(sentences = tweets, total_examples = w2v_model.corpus_count, epochs = 100)

(7166444, 10614100)

### Now: Explore your model!

In [123]:
print(w2v_model.wv.most_similar(positive = ["germany"]))
print(w2v_model.wv.most_similar(positive = ["clinton"]))
print(w2v_model.wv.most_similar(positive = ["democrats"]))
print(w2v_model.wv.most_similar(positive = ["mexico"]))
print(w2v_model.wv.most_similar(positive = ["china"]))
print(w2v_model.wv.most_similar(positive = ["mexico", "trade"], negative = ["wall"]))

[('biarritz', 0.4658207893371582), ('round', 0.3740491271018982), ('nato', 0.37097910046577454), ('countries', 0.35524648427963257), ('warren', 0.34194955229759216), ('european', 0.33061978220939636), ('likewise', 0.3289884030818939), ('gps', 0.3195797801017761), ('7', 0.3121258616447449), ('fusion', 0.3114713728427887)]
[('crooked', 0.5022450685501099), ('deleted', 0.40662604570388794), ('hillary', 0.3783566951751709), ('foundation', 0.33433467149734497), ('russians', 0.33428552746772766), ('me', 0.3295871615409851), ('campaign', 0.3278309106826782), ('ed', 0.32764503359794617), ('wins', 0.32643961906433105), ('phony', 0.3143817186355591)]
[('dems', 0.7297271490097046), ('they', 0.4946945309638977), ('democrat', 0.4721542000770569), ('we', 0.43369194865226746), ('people', 0.3562706708908081), ('losers', 0.33712688088417053), ('you', 0.3129364550113678), ('criminals', 0.25872164964675903), ('right', 0.25418299436569214), ('drugs', 0.253610223531723)]
[('china', 0.3199893832206726), ('f

### Explore the possibilities the model by e.g. switching from skip-gram to cbow, using averaging instead of concatenation, chosing a larger embedding size, more negative examples, etc.

### Try using ``gensim.models.phrases`` in order to form bigrams

In [167]:
from gensim.models.phrases import Phrases, Phraser

phrases = Phrases(tweets, min_count=100, threshold=10)
bigram = Phraser(phrases)

In [172]:
sorted(list(bigram.phrasegrams.items()), reverse = True)

[((b'\xe2\x80\x99', b't'), 40.14282388966591),
 ((b'\xe2\x80\x99', b's'), 39.051824673367356),
 ((b'witch', b'hunt'), 24.661771250556296),
 ((b'will', b'be'), 15.202834497541446),
 ((b'united', b'states'), 98.71593416819549),
 ((b'thank', b'you'), 49.87788116523749),
 ((b'our', b'country'), 27.60977332033378),
 ((b'fake', b'news'), 74.46145685997172),
 ((b'don', b'\xe2\x80\x99'), 16.771136693276688)]

In [176]:
tweets = list(bigram[tweets])

In [193]:
w2v_model.wv["clinton"]

array([-2.9408352 , -2.9448547 , -0.0644585 ,  2.5617635 ,  0.11626738,
        0.9916419 , -0.982047  , -0.36746535,  2.282364  ,  4.807505  ,
       -0.57372135, -0.1776367 ,  3.6928544 , -0.07633026, -0.3486168 ,
       -1.1801438 , -2.0244286 ,  3.9215038 ,  5.7360845 ,  1.1472751 ,
       -0.6600361 ,  3.4406743 ,  0.42411965,  2.2777426 ,  4.44871   ,
       -0.51950186, -3.310043  , -0.11698956,  1.4302472 , -1.8335285 ,
       -1.2929436 , -3.6467178 ,  0.5088786 ,  0.97771716, -2.7727382 ,
       -1.151348  , -2.8728178 ,  0.28081998,  0.15078373,  0.28773436,
        2.4083438 ,  1.5199484 ,  0.94393754, -3.9986844 , -0.9181879 ,
        2.4555998 , -0.99208266,  0.16184539, -2.7211847 , -0.95676094,
       -1.9936352 ,  0.18663087,  0.7953555 , -1.7466047 ,  1.345637  ,
        0.13392718, -0.39346752, -1.2971301 ,  1.736669  ,  1.0345834 ,
        3.0065439 , -0.919131  , -0.42990413,  1.9712603 , -1.983691  ,
       -0.59394854, -1.0466216 ,  2.0349941 ,  2.022733  , -0.22